# Interactive Plot
Interactive plot to model Covid-19 in Indonesia. This repository is part of Covid-19 Modelling Project by [Department of Electrical and Information Engineering Universitas Gadjah Mada](http://jteti.ugm.ac.id/index.php?ver=YQ%3D%3D=). Interactive site using Voilà can be used using (need one to five minutes to load the plot):

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/yasirroni/Indonesia-Covid-Model/master?urlpath=%2Fvoila%2Frender%2FInteractive%2520Plot.ipynb)

In [1]:
from IPython.display import display
from scipy.integrate import odeint
from datetime import datetime as dt

import ipywidgets as widgets
import numpy as np
import bqplot as bq
import pandas as pd
import threading
import time

In [2]:
# flag to control loop
flag_on = True
flag_init_input = False
flag_add_checkpoint = False
flag_compute = False

# init data to plot (dummy)
def init(t='2020-01-01', S=0, I=0, R=0, D=0):
    global t_list, S_list, I_list, R_list, D_list
    t_list = [t]
    S_list = [S]
    I_list = [I]
    R_list = [R]
    D_list = [D]
init()

In [3]:
# initial input
S_init = widgets.IntText(
    value=1e7,
    description='S initial:',
    disabled=False
)

I_init = widgets.IntText(
    value=100,
    description='I initial:',
    disabled=False
)

R_init = widgets.IntText(
    value=1,
    description='R initial:',
    disabled=False
)
D_init = widgets.IntText(
    value=1,
    description='D initial:',
    disabled=False
)

# initial date
initial_date = widgets.DatePicker(
    description = 'Start Date:',
    disabled = False,
    value = dt.date(dt.strptime('01/03/20', '%d/%m/%y'))
)

# initial button
button_initial = widgets.Button(
    description = 'ENTER',
    button_style = 'primary',
    layout = widgets.Layout(width='100px')
)

def button_initial_click(self):
    global flag_init_input
    flag_init_input = True

button_initial.on_click(button_initial_click)

# initial wrapper
initials = widgets.VBox(
    children = (initial_date, S_init, I_init, R_init, D_init, button_initial),
)

In [4]:
# checkpoint
checkpoint_date = widgets.DatePicker(
    description='Checkpoint:',
    disabled=False,
    value = dt.date(dt.strptime('31/03/21', '%d/%m/%y'))
)

beta = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=10.0,
    step=0.0001,
    description='Beta:',
    disabled=False
)
gamma = widgets.BoundedFloatText(
    value=0.05,
    min=0,
    max=10.0,
    step=0.0001,
    description='Gamma:',
    disabled=False
)
mu = widgets.BoundedFloatText(
    value=0.005,
    min=0,
    max=10.0,
    step=0.0001,
    description='Mu:',
    disabled=False
)

button_checkpoint = widgets.Button(
    description = 'ADD',
    button_style = 'primary',
    layout = widgets.Layout(width='100px')
)

def button_checkpoint_click(self):
    global flag_add_checkpoint
    flag_add_checkpoint = False

button_checkpoint.on_click(button_checkpoint_click)

# checkpoint wrapper
checkpoints = widgets.VBox(
    children = (checkpoint_date, beta, gamma, mu, button_checkpoint)
)

In [5]:
# stop button
button_stop = widgets.Button(
    description = 'TURN OFF',
    icon = 'fa-toggle-on',
    button_style = 'success',
    layout = widgets.Layout(width='200px')
)

def button_stop_click(self):
    global flag_on
    global thread
    if flag_on:
        flag_on = False
        button_stop.description = 'TURN ON'
        button_stop.icon = 'fa-toggle-off'
        button_stop.button_style = ''
    else:
        flag_on = True
        button_stop.description = 'TURN OFF'
        button_stop.icon = 'fa-toggle-on'
        button_stop.button_style = 'success'
        
        # create thread
        thread = threading.Thread(target=work)
        
        # start thread
        thread.start()

button_stop.on_click(button_stop_click)

In [6]:
# scale
x_sc = bq.DateScale()
y_sc_lin = bq.LinearScale()
y_sc_log = bq.LogScale()

# axis
x_ax = bq.Axis(
    label = 't',
    scale = x_sc
)
y_ax_lin = bq.Axis(
    label = 'y(t)',
    scale = y_sc_lin,
    orientation = 'vertical'
)
y_ax_log = bq.Axis(
    label = 'y(t)',
    scale = y_sc_log,
    orientation = 'vertical'
)

# line
lines_lin = bq.Lines(
    x = t_list,
    y = [S_list, I_list, R_list, D_list],
    colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728'],
    labels = ['S', 'I', 'R', 'D'],
    display_legend = True,
    scales = {
        'x': x_sc,'y': y_sc_lin
    }
)
lines_log = bq.Lines(
    x = t_list,
    y = [S_list, I_list, R_list, D_list],
    colors = ['#2ca02c', '#ff7f0e', '#1f77b4', '#d62728'],
    labels = ['S', 'I', 'R', 'D'],
    display_legend = True,
    scales = {
        'x': x_sc,'y': y_sc_log
    }
)

# figure
fig_lin = bq.Figure(
    layout = widgets.Layout(width='500px', height='310px'),
    axes = [x_ax, y_ax_lin],
    marks = [lines_lin],
    fig_margin = dict(top=10, bottom=40, left=50, right=10),
    background_style = {'fill': 'white'}
)
fig_log = bq.Figure(
    layout = widgets.Layout(width='500px', height='310px'),
    axes = [x_ax, y_ax_log],
    marks = [lines_log],
    fig_margin = dict(top=10, bottom=40, left=50, right=10),
    background_style = {'fill': 'white'}
)

# checkbox
checkbox_S = widgets.Checkbox(
    value=True,
    description='S',
    disabled=False,
    indent=False
)
checkbox_I = widgets.Checkbox(
    value=True,
    description='I',
    disabled=False,
    indent=False
)
checkbox_R = widgets.Checkbox(
    value=True,
    description='R',
    disabled=False,
    indent=False
)
checkbox_D = widgets.Checkbox(
    value=True,
    description='D',
    disabled=False,
    indent=False
)
checkbox_log = widgets.Checkbox(
    value=False,
    description='Log Scale',
    disabled=False,
    indend=False
)
checkbox_lin = widgets.Checkbox(
    value=True,
    description='Linear Scale',
    disabled=False,
    indend=False
)

checboxes = widgets.HBox(
#     children = (checkbox_S, checkbox_I, checkbox_R, checkbox_D),# checkbox_log),
    children = (checkbox_S, checkbox_I, checkbox_R, checkbox_D, checkbox_lin, checkbox_log),
    layout = widgets.Layout(width='500px')
)

plots = widgets.VBox(
    children = (fig_lin, checboxes),
#     layout = widgets.Layout(border = 'solid 2px gray', width='530px')
)

In [7]:
# app_wrapper
app_input = widgets.VBox(
    children = (initials,),
#     layout = widgets.Layout(border = 'solid 2px gray', width='344px')
)

app_modifier = checkpoints
app_screen = plots
app_interactive = widgets.HBox(
    children = (app_modifier, app_screen),
#     layout = widgets.Layout(border = 'solid 2px gray', width='878px')
)

In [8]:
def func_ode(y, t, beta, gamma, mu):
    [S, I, R, D] = y
    
    N = S + I + R + D
    
    dSdt = - (beta * S * I / N)
    dIdt = (beta * S * I / N) - ((gamma + mu) * I)
    dRdt = gamma*I
    dDdt = mu*I
    
    return [dSdt, dIdt, dRdt, dDdt]

def work():
    global flag_on, flag_init_input
    global t_list, S_list, I_list, R_list, D_list
    
    checkbox_log_prev = checkbox_log.value
    checkbox_lin_prev = checkbox_lin.value
    
    while flag_on:
        # reset and init
        if flag_init_input:           
            init(str(initial_date.value), S_init.value, I_init.value, R_init.value, D_init.value)
            t_start = initial_date.value
            flag_init_input = False
        
        # make t
        t_len = int((checkpoint_date.value - initial_date.value).days) + 1 # the "+ 1" is to include endpoint
        t_new = pd.to_datetime(
            np.linspace(
                pd.Timestamp(initial_date.value).value, 
                pd.Timestamp(checkpoint_date.value).value, 
                t_len,
                endpoint=True
            )
        )       
        t_dummy = np.linspace(1, t_len, t_len)
        
        # make ode
        ode_out = odeint(
            func_ode,
            [S_init.value, I_init.value, R_init.value, D_init.value],
            t_dummy,
            args=(beta.value, gamma.value, mu.value)
        )
    
        # unpack ode_out
        S_new = ode_out[:,0]
        I_new = ode_out[:,1]
        R_new = ode_out[:,2]
        D_new = ode_out[:,3]
        
        # save checkpoint
        
        # merge and update plot
        t_list = t_new.tolist()
        S_list = S_new.tolist()
        I_list = I_new.tolist()
        R_list = R_new.tolist()
        D_list = D_new.tolist()
               
        # y_axes
        if checkbox_log_prev != checkbox_log.value or checkbox_lin_prev != checkbox_lin.value:
            if checkbox_log.value and checkbox_lin.value:
                plots.children = (fig_lin, fig_log, checboxes)
            elif checkbox_log.value:
                plots.children = (fig_log, checboxes)
            elif checkbox_lin.value:
                plots.children = (fig_lin, checboxes)
            else:
                plots.children = (checboxes,)
            
            checkbox_log_prev = checkbox_log.value
            checkbox_lin_prev = checkbox_lin.value

        # update plot
        y_temp = []
        labels_temp = []
        colors_temp = []
        if checkbox_S.value:
            y_temp.append(S_list)
            labels_temp.append('S')
            colors_temp.append('#2ca02c')
        if checkbox_I.value:
            y_temp.append(I_list)
            labels_temp.append('I')
            colors_temp.append('#ff7f0e')
        if checkbox_R.value:
            y_temp.append(R_list)
            labels_temp.append('R')
            colors_temp.append('#1f77b4')
        if checkbox_D.value:
            y_temp.append(D_list)
            labels_temp.append('D')
            colors_temp.append('#d62728')
        if checkbox_lin:
            lines_lin.y = y_temp
            lines_lin.labels = labels_temp
            lines_lin.x = t_list
            lines_lin.colors = colors_temp
        if checkbox_log:
            lines_log.y = y_temp
            lines_log.labels = labels_temp
            lines_log.x = t_list
            lines_log.colors = colors_temp

        time.sleep(0.05)

In [9]:
# set the flag to true
flag_on = True

# create thread
thread = threading.Thread(target=work)

# start thread
thread.start()

In [10]:
# display app_input
display(app_input)

In [11]:
# display app_interactive
display(app_interactive)

In [12]:
button_stop

Button(button_style='success', description='TURN OFF', icon='toggle-on', layout=Layout(width='200px'), style=B…